## Câu 1: viết câu lệnh sql

In [75]:
import sqlite3
# Câu SQL tính hệ số tương quan Pearson
query = """
SELECT 
    (COUNT(*) * SUM(A * B) - SUM(A) * SUM(B)) AS numerator,
    SQRT(COUNT(*) * SUM(A * A) - SUM(A) * SUM(A)) AS denominator_a,
    SQRT(COUNT(*) * SUM(B * B) - SUM(B) * SUM(B)) AS denominator_b
FROM data
"""

## Câu 2: thử nghiệm và chuyển đổi dữ liệu

In [1]:
import sqlite3
import pandas as pd
conn = sqlite3.connect("abc.db")
cursor = conn.cursor()


In [2]:
#Chuyển đổi thành dạng quan hệ
cursor.execute("drop table if exists Eto_quanhe")

cursor.execute('''
                    create table Eto_quanhe(
                    ngay TEXT,
                    Êtô TEXT,
                    Diem REAL)

''')

conn.commit()
cursor.executescript('''
INSERT INTO Eto_quanhe (ngay, Êtô, Diem) VALUES
('day 1', 'A', 8),
('Day 1', 'B', 9),
('Day 1', 'C', 7),
('Day 2', 'A', 7.5),
('Day 2', 'B', 8.5),
('Day 2', 'C', 7),
('Day 3', 'A', 6),
('Day 3', 'B', 7),
('Day 3', 'C', 8),
('Day 4', 'A', 7),
('Day 4', 'B', 6),
('Day 4', 'C', 5);
''')

conn.commit()

In [3]:
cursor.execute("DROP VIEW IF EXISTS Eto_chatluong")

cursor.execute('''
CREATE VIEW Eto_chatluong AS
SELECT
    ngay,
    Êtô,
    Diem,
    CASE
        WHEN Diem < 5 THEN 'Kém'
        WHEN Diem >= 5 AND Diem < 7 THEN 'Trung bình'
        WHEN Diem >= 7 AND Diem < 9 THEN 'Tốt'
        ELSE 'Rất tốt'
    END AS chatluong
FROM  Eto_quanhe;
''')
cursor.execute('''
SELECT  Êtô, chatluong, COUNT(*) as count
FROM Eto_chatluong
GROUP BY  Êtô, chatluong
ORDER BY  Êtô, chatluong;

''')

conn.commit()

rows = cursor.fetchall()
for row in rows:
    print(row)

conn.commit()

('A', 'Trung bình', 1)
('A', 'Tốt', 3)
('B', 'Rất tốt', 1)
('B', 'Trung bình', 1)
('B', 'Tốt', 2)
('C', 'Trung bình', 1)
('C', 'Tốt', 3)


In [4]:
import pandas as pd

df = pd.read_sql_query('''
SELECT Êtô, chatluong, COUNT(*) as soluong
FROM  Eto_chatluong
GROUP BY Êtô, chatluong
''', conn)

print(df)

  Êtô   chatluong  soluong
0   A  Trung bình        1
1   A         Tốt        3
2   B     Rất tốt        1
3   B  Trung bình        1
4   B         Tốt        2
5   C  Trung bình        1
6   C         Tốt        3


In [5]:
from scipy.stats import chi2_contingency

# Pivot để có ma trận tần suất
table = df.pivot(index='Êtô', columns='chatluong', values='soluong').fillna(0)
print(table)

chi2, p, dof, expected = chi2_contingency(table)
print("Chi-squared:", chi2)
print("p-value:", p)


chatluong  Rất tốt  Trung bình  Tốt
Êtô                                
A              0.0         1.0  3.0
B              1.0         1.0  2.0
C              0.0         1.0  3.0
Chi-squared: 2.25
p-value: 0.6898864931364932


Kết quả kiểm định Chi-squared: Giá trị Chi-squared = 2.25

p-value = 0.6899

Vì p-value lớn hơn 0.05, ta không có đủ bằng chứng thống kê để bác bỏ giả thuyết không (H₀), tức là:

Không có sự khác biệt đáng kể về điểm số giữa các mẫu xe

Kết luận: Không có mẫu xe nào nổi bật

Phân phối điểm số không phụ thuộc vào loại xe

## Câu 3
### Bảng flights(departure_time,…) chứa các giá trị thời gian dưới dạng số nguyên (ví dụ: 830 cho 8:30 AM, 1445 cho 2:45 PM). Hãy chuyển đổi các giá trị này thành định dạng thời gian

In [81]:
import sqlite3
import pandas as pd

# Tạo kết nối SQLite
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng flights
cursor.execute("""
CREATE TABLE flights (
    id INTEGER PRIMARY KEY,
    departure_time INTEGER
)
""")

# Thêm dữ liệu giả lập
data = [(1, 830), (2, 1445), (3,123 ), (4,234 ), (5, 456)]
cursor.executemany("INSERT INTO flights (id, departure_time) VALUES (?, ?)", data)
conn.commit()

In [82]:
query = """
SELECT 
    id,
    departure_time,
    printf('%02d:%02d', departure_time / 100, departure_time % 100) AS departure_time_formatted
FROM flights
"""
df = pd.read_sql_query(query, conn)
print(df)

   id  departure_time departure_time_formatted
0   1             830                    08:30
1   2            1445                    14:45
2   3             123                    01:23
3   4             234                    02:34
4   5             456                    04:56


## Câu 4: 
### Viết truy vấn SQL để tìm các ngoại lệ bằng cách sử dụng MAD. Một quy tắc chung là xem xét các giá trị ngoại lệ lớn hơn 1,5 lần so với giá trị MAD, trong đó x là số độ lệch chuẩn mà ta coi là có ý nghĩa.


In [26]:
cursor.execute("DROP TABLE IF EXISTS data")
cursor.execute("CREATE TABLE data(val REAL)")

#  Chèn dữ liệu mẫu
values = [(0,), (126,), (132,), (12,), (101,), (200,), (12,), (13,), (191,), (10,), (200,)]
cursor.executemany("INSERT INTO data(val) VALUES (?)", values)
conn.commit()

#  Tính med
cursor.execute("""
SELECT val FROM data ORDER BY val
""")
vals = [row[0] for row in cursor.fetchall()]
n = len(vals)
median = (vals[n // 2] if n % 2 == 1 else (vals[n // 2 - 1] + vals[n // 2]) / 2)

# Tính |xi - median|
abs_devs = [abs(v - median) for v in vals]
abs_devs.sort()
mad = (abs_devs[n // 2] if n % 2 == 1 else (abs_devs[n // 2 - 1] + abs_devs[n // 2]) / 2)

# Chọn các giá trị là ngoại lệ (vượt 1.5 × MAD)
threshold = 1.5 * mad
outliers = [v for v in vals if abs(v - median) > threshold]

#  Xuất kết quả
print(f"Median = {median}")
print(f"MAD = {mad}")
print(f"Ngưỡng ngoại lệ (1.5×MAD): ±{threshold}")
print("Giá trị ngoại lệ:", outliers)

Median = 101.0
MAD = 89.0
Ngưỡng ngoại lệ (1.5×MAD): ±133.5
Giá trị ngoại lệ: []


## Câu 5: 
### Hãy xác định liệu hai người trong bảng Patient(last_name, weight, height) có phải là một người hay không bằng cách sử dụng khoảng cách kết hợp Boolean trên “last_name” và “weight”.

In [ ]:
cursor.execute("DROP TABLE IF EXISTS Patient")
cursor.execute("""
CREATE TABLE Patient (
    id INTEGER PRIMARY KEY,
    last_name TEXT,
    weight REAL,
    height REAL
)
""")

#  Thêm dữ liệu mẫu
patients = [
    (1, 'Nguyen', 60, 170),
    (2, 'Nguyen', 60, 171),  
    (3, 'Tran', 58, 165),
    (4, 'Nguyen', 65, 170),
    (5, 'Le', 70, 175),
    (6, 'Nguyen', 60, 170),  
    (7, 'Tran', 58, 165),
    (8, 'Nguyen', 65, 170),
    (9, 'Le', 70, 175),
    (10, 'Nguyen', 60, 170),  
    (11, 'Tran', 58, 165),
    (12, 'Nguyen', 65, 170) 
]

cursor.executemany("INSERT INTO Patient VALUES (?, ?, ?, ?)", patients)
conn.commit()

#  Truy vấn kết hợp từng cặp và tính "khoảng cách Boolean"
cursor.execute("""
SELECT 
    p1.id AS id1,
    p2.id AS id2,
    p1.last_name = p2.last_name AS same_last_name,
    p1.weight = p2.weight AS same_weight,
    (p1.last_name = p2.last_name AND p1.weight = p2.weight) AS is_same_person
FROM Patient p1
JOIN Patient p2 ON p1.id < p2.id
""")
for i in cursor:
    print(i)

(1, 2, 1, 1, 1)
(1, 3, 0, 0, 0)
(1, 4, 1, 0, 0)
(1, 5, 0, 0, 0)
(1, 6, 1, 1, 1)
(1, 7, 0, 0, 0)
(1, 8, 1, 0, 0)
(1, 9, 0, 0, 0)
(1, 10, 1, 1, 1)
(1, 11, 0, 0, 0)
(1, 12, 1, 0, 0)
(2, 3, 0, 0, 0)
(2, 4, 1, 0, 0)
(2, 5, 0, 0, 0)
(2, 6, 1, 1, 1)
(2, 7, 0, 0, 0)
(2, 8, 1, 0, 0)
(2, 9, 0, 0, 0)
(2, 10, 1, 1, 1)
(2, 11, 0, 0, 0)
(2, 12, 1, 0, 0)
(3, 4, 0, 0, 0)
(3, 5, 0, 0, 0)
(3, 6, 0, 0, 0)
(3, 7, 1, 1, 1)
(3, 8, 0, 0, 0)
(3, 9, 0, 0, 0)
(3, 10, 0, 0, 0)
(3, 11, 1, 1, 1)
(3, 12, 0, 0, 0)
(4, 5, 0, 0, 0)
(4, 6, 1, 0, 0)
(4, 7, 0, 0, 0)
(4, 8, 1, 1, 1)
(4, 9, 0, 0, 0)
(4, 10, 1, 0, 0)
(4, 11, 0, 0, 0)
(4, 12, 1, 1, 1)
(5, 6, 0, 0, 0)
(5, 7, 0, 0, 0)
(5, 8, 0, 0, 0)
(5, 9, 1, 1, 1)
(5, 10, 0, 0, 0)
(5, 11, 0, 0, 0)
(5, 12, 0, 0, 0)
(6, 7, 0, 0, 0)
(6, 8, 1, 0, 0)
(6, 9, 0, 0, 0)
(6, 10, 1, 1, 1)
(6, 11, 0, 0, 0)
(6, 12, 1, 0, 0)
(7, 8, 0, 0, 0)
(7, 9, 0, 0, 0)
(7, 10, 0, 0, 0)
(7, 11, 1, 1, 1)
(7, 12, 0, 0, 0)
(8, 9, 0, 0, 0)
(8, 10, 1, 0, 0)
(8, 11, 0, 0, 0)
(8, 12, 1, 1, 1)
(9, 10, 0, 0, 0)